In [1]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point
import requests
from collections import defaultdict

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_64193/472444752.py:3: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, 

In [2]:
# Your property data
property_metadata = pd.read_csv("../data/raw/properties.csv")

# Convert the coordinates column to a suitable format (assuming it's a string like '[lat, lon]')
property_metadata['coordinates'] = property_metadata['coordinates'].apply(eval)

# Create points and names from the property_metadata
coords = [Point(xy[1], xy[0]) for xy in property_metadata['coordinates']]
names = property_metadata['name'].tolist()
cost = property_metadata['weekly_rent'].tolist()
beds = property_metadata['beds'].tolist()
baths = property_metadata['baths'].tolist()
parkings = property_metadata['parkings'].tolist()

# Convert to GeoDataFrame
property = gpd.GeoDataFrame({'name': names, 'geometry': coords, 'cost': cost, 'beds': beds, 'baths': baths, 'parkings': parkings})


In [3]:
# read into shape file
sf = gpd.read_file("../data/statistical_area_level2/SA2_2021_AUST_GDA2020.shp")
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
victoria_gdf = sf[sf['STE_NAME21'] == 'Victoria']
victoria_gdf.to_file('../data/raw/victoria.geojson', driver='GeoJSON')

In [4]:
# Perform a spatial join
property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_64193/3292555783.py:2: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs + ...

  property_SA2 = gpd.sjoin(property, victoria_gdf, how="inner", op="within")


In [5]:
selected_columns = ['name', 'geometry', 'cost', 'beds', 'baths', 'parkings', 'SA2_CODE21', 'SA2_NAME21']
property_SA2 = property_SA2[selected_columns]

In [6]:
# read into PTV station dataset
PTV_metro = gpd.read_file('../data/external_SA2/PTV/PTV_metro/PTV_METRO_TRAIN_STATION.shp')
PTV_regional = gpd.read_file("../data/external_SA2/PTV/PTV_regional/PTV_REGIONAL_TRAIN_STATION.shp")

In [7]:
PTV_station = PTV_metro.append(PTV_regional, ignore_index=True)
PTV_station = PTV_station.drop_duplicates(subset='STOP_NAME', keep='first')

/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_64193/1230334281.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  PTV_station = PTV_metro.append(PTV_regional, ignore_index=True)


In [8]:
def euclidean_distance(point1, point2):
    """
    finding out the euclidean distance between two points
    """
    return ((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2) ** 0.5

def get_distance(first_point, second_point):
    """
    get the driving distance by ORS
    """
    ORS_BASE_URL = "http://localhost:8080/ors/v2/directions/driving-car"
    url = f'{ORS_BASE_URL}?start={first_point[0]},{first_point[1]}&end={second_point[0]},{second_point[1]}'
    response = requests.get(url)
    json_response = response.json()
    
    if 'error' not in json_response:
        try:
            return json_response['features'][0]['properties']['summary']['duration'] / 60
        except KeyError: 
            return float('inf')
    return float('inf')


def find_proximity(property_df, facility_df, name):
    """
    find out the closest facilities and add them into columns
    """
    # This will hold the name of the nearest facility for each property
    nearest_facilities = []

    # Loop through properties
    for idx1, prop_row in property_df.iterrows():
        distances = {}  # Using a dictionary to map distances to facility point
        prop_point = prop_row['geometry'].coords[0]
        
        # Loop through facilities
        for idx2, fac_row in facility_df.iterrows():
            fac_point = fac_row['geometry'].coords[0]
            # Compute the Euclidean distance
            dist = euclidean_distance(prop_point, fac_point)
            distances[dist] = fac_point

        # Finding the facility name corresponding to the shortest distance
        min_distance = min(distances.keys())
        nearest_facilities.append(get_distance(prop_point, distances[min_distance]))

    property_df['Nearest_{}'.format(name)] = nearest_facilities

    return property_df

In [9]:
# find out the closest station
property_station = find_proximity(property_SA2, PTV_station, 'station')

In [10]:
property_station.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.486667
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.295000
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,0.0,206041503,Melbourne CBD - East,5.560000
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,2.920000
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,1.075000


In [11]:
park = gpd.read_file('../data/external_SA2/Park.geojson')

In [12]:
property_station_park = find_proximity(property_station, park, 'park')

In [13]:
property_station_park.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.486667,4.193333
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.295000,6.461667
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,0.0,206041503,Melbourne CBD - East,5.560000,6.661667
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,2.920000,3.980000
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,1.075000,6.253333


In [14]:
shop = gpd.read_file('../data/external_SA2/Mall, Shopping Centre & Department Store.geojson')

In [15]:
property_station_park_shop = find_proximity(property_station_park, shop, 'shop')

In [16]:
property_station_park_shop.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park,Nearest_shop
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.486667,4.193333,1.875000
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.295000,6.461667,3.020000
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,0.0,206041503,Melbourne CBD - East,5.560000,6.661667,3.220000
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,2.920000,3.980000,2.151667
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,1.075000,6.253333,2.813333


In [17]:
hospital = gpd.read_file('../data/external_SA2/Hospital.geojson')

In [18]:
property_station_park_shop_hospital = find_proximity(property_station_park_shop, hospital, 'hospital')

In [19]:
property_station_park_shop_hospital.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park,Nearest_shop,Nearest_hospital
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.486667,4.193333,1.875000,5.448333
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.295000,6.461667,3.020000,6.071667
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,0.0,206041503,Melbourne CBD - East,5.560000,6.661667,3.220000,6.835000
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,2.920000,3.980000,2.151667,5.170000
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,1.075000,6.253333,2.813333,5.890000


In [20]:
school_df = pd.read_csv("../data/external_SA2/complete_school_location.csv", encoding='latin-1')
school_df['coordinates'] = school_df.apply(lambda row: [row['Y'], row['X']], axis=1)

In [21]:
school_df.head()

,geometry,X,Y,School_Type,School_Name,Postal_Town,coordinates
0,POINT (145.3643874 -38.084432),145.364387,-38.084432,college,Hillcrest Christian College (Ayr Hill Campus),NaN,"[-38.084432, 145.3643874]"
1,POINT (-96.9814539 28.8155588),-96.981454,28.815559,college,Victoria College,NaN,"[28.8155588, -96.9814539]"
2,POINT (143.858787 -37.5611479),143.858787,-37.561148,college,Arts Academy,NaN,"[-37.5611479, 143.858787]"
3,POINT (144.972154 -37.8103129),144.972154,-37.810313,university,James Cook University,NaN,"[-37.8103129, 144.972154]"
4,POINT (145.1490967 -37.8768613),145.149097,-37.876861,college,Huanya Cultural Training Centre,NaN,"[-37.8768613, 145.1490967]"


In [22]:
school_metadata = school_df

# Create points and names from the school_metadata
coords = [Point(xy[1], xy[0]) for xy in school_metadata['coordinates']]
names = school_metadata['School_Type'].tolist()

# Convert to GeoDataFrame
school = gpd.GeoDataFrame({'name': names, 'geometry': coords})

In [23]:
school_SA2 = gpd.sjoin(school, victoria_gdf, how="inner", op="within")

/Users/henrik_hao/opt/anaconda3/envs/MAST30034/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/var/folders/hl/0j63l0bd7n3ggf_3ylf9n8q80000gn/T/ipykernel_64193/1493813974.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: +proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs + ...

  school_SA2 = gpd.sjoin(school, victoria_gdf, how="inner", op="within")


In [24]:
school_SA2.head()

,name,geometry,index_right,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21
0,college,POINT (145.36439 -38.08443),1007,212031555,Clyde North - North,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,6.8833,http://linked.data.gov.au/dataset/asgsed3/SA2/...
1082,Primary,POINT (145.37283 -38.08273),1007,212031555,Clyde North - North,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,6.8833,http://linked.data.gov.au/dataset/asgsed3/SA2/...
2062,Pri/Sec,POINT (145.36380 -38.08468),1007,212031555,Clyde North - North,1,New,21203,Casey - South,212,Melbourne - South East,2GMEL,Greater Melbourne,2,Victoria,AUS,Australia,6.8833,http://linked.data.gov.au/dataset/asgsed3/SA2/...
2,college,POINT (143.85879 -37.56115),645,201011002,Ballarat,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,12.3787,http://linked.data.gov.au/dataset/asgsed3/SA2/...
82,Primary,POINT (143.85394 -37.56401),645,201011002,Ballarat,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,12.3787,http://linked.data.gov.au/dataset/asgsed3/SA2/...


In [25]:
property_station_park_shop_hospital_school = find_proximity(property_station_park_shop_hospital, school_SA2, 'school')

In [26]:
property_station_park_shop_hospital_school.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park,Nearest_shop,Nearest_hospital,Nearest_school
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.486667,4.193333,1.875000,5.448333,0.831667
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.295000,6.461667,3.020000,6.071667,1.430000
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,0.0,206041503,Melbourne CBD - East,5.560000,6.661667,3.220000,6.835000,0.761667
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,2.920000,3.980000,2.151667,5.170000,1.238333
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,1.075000,6.253333,2.813333,5.890000,1.223333


In [27]:
supermarket = gpd.read_file('../data/external_SA2/Supermarket.geojson')

In [28]:
property_station_park_shop_hospital_school_supermarket = find_proximity(property_station_park_shop_hospital_school, supermarket, 'supermarket')

In [29]:
property_station_park_shop_hospital_school_supermarket.head()

,name,geometry,cost,beds,baths,parkings,SA2_CODE21,SA2_NAME21,Nearest_station,Nearest_park,Nearest_shop,Nearest_hospital,Nearest_school,Nearest_supermarket
0,904/265 Exhibition Street Melbourne VIC 3000,POINT (144.96912 -37.80951),850.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.486667,4.193333,1.875000,5.448333,0.831667,0.743333
6,2302/17 Spring Street Melbourne VIC 3000,POINT (144.97440 -37.81475),1500.0,2.0,2.0,1.0,206041503,Melbourne CBD - East,1.295000,6.461667,3.020000,6.071667,1.430000,2.190000
12,11/187 Collins Street Melbourne VIC 3000,POINT (144.96785 -37.81540),800.0,1.0,1.0,0.0,206041503,Melbourne CBD - East,5.560000,6.661667,3.220000,6.835000,0.761667,2.823333
19,1605/68 La Trobe Street Melbourne VIC 3000,POINT (144.96679 -37.80836),590.0,2.0,1.0,1.0,206041503,Melbourne CBD - East,2.920000,3.980000,2.151667,5.170000,1.238333,2.193333
21,3007/35 Spring Street Melbourne VIC 3000,POINT (144.97400 -37.81417),2300.0,3.0,2.0,2.0,206041503,Melbourne CBD - East,1.075000,6.253333,2.813333,5.890000,1.223333,1.968333


In [30]:
property_station_park_shop_hospital_school_supermarket.to_csv("../data/curated/properties_proximity.csv")